In [1]:
import pandas as pd
from itertools import product
from sklearn.preprocessing import OneHotEncoder, StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

import numpy as np

from tqdm import tqdm
import matplotlib.pyplot as plt
import copy

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

In [4]:
class DataFinal():
    def __init__(self):
        self.original = pd.read_csv('../html2023-spring-final-project/train.csv')
        self.get_labels()
        data = self.dataPreprocessing(self.original)
        self.createEncodeDataTraining(data)
        self.splitDataset()

    def get_labels(self):
        self.labelsOriginal = self.original['Danceability'].to_numpy()
        self.original.drop(['Danceability'], axis=1, inplace=True)

        # print(self.labelsOriginal) 

        encoderOutput = OneHotEncoder()
        encodedLabel = encoderOutput.fit_transform(self.labelsOriginal.reshape(-1, 1))
        self.labels =  encodedLabel.toarray()


    def fillOptions(self, data, option = 'max'):
        if data.isna().sum() != len(data):
            if option == 'max':
                return data.value_counts().idxmax()
            elif option == 'mean':
                return data.mean()
            elif option == 'median':
                return data.median()
            
    def filterArtistComposerDance(self, data, nameColumnFill):

        listArtist = data['Artist'].unique()
        listComposer = data['Composer'].unique()
    
        filter = list(product(listArtist, listComposer))

        newData = pd.DataFrame(columns=data.columns)

        for i in filter:
            artist, composer = i[0], i[1]
            filterData = data[(data['Artist'] == artist) & (data['Composer'] == composer) ].copy()
            if len(filterData) != 0:
            # Fill column Name
                for nameColumn in nameColumnFill:
                    fillInfo = self.fillOptions(filterData[nameColumn], option = 'max')
                    if fillInfo != None:
                        filterData.loc[:,nameColumn].fillna(fillInfo, inplace=True)

                newData = pd.concat([newData, filterData], ignore_index=True)
        return newData
    
    def filterArtist(self, data, nameColumnFill):
        listDance = data['Artist'].unique()
        filter = listDance


        newData = pd.DataFrame(columns=data.columns)

        for i in filter:
            dance = i
            filterData = data[ (data['Artist'] == dance)].copy()
    
    
            if len(filterData) != 0:
                # Fill column Name
                for nameColumn in nameColumnFill:
                    fillInfo = self.fillOptions(filterData[nameColumn], option = 'max')
                    if fillInfo != None:
                        filterData.loc[:,nameColumn].fillna(fillInfo, inplace=True)

                newData = pd.concat([newData, filterData], ignore_index=True)

        return newData
    
    def filterFillData(self, data, nameColumnFill):
        if not data.isnull().any().any():
            return data
        else:
            for nameColumn in nameColumnFill:
                fillInfo = self.fillOptions(data[nameColumn], option = 'max')
                if fillInfo != None:
                    data.loc[:,nameColumn].fillna(fillInfo, inplace=True)
        return data
    
    def dataPreprocessing(self, original):
    
        # pd.options.mode.chained_assignment = None

        data = original.copy()

        nameColumnFill = ['Energy', 'Key', 'Loudness', 'Speechiness', 'Acousticness', 'Instrumentalness', 'Liveness', 'Valence', 'Tempo', 'Duration_ms', 'Duration_ms', 'Views', 'Likes', "Stream" , "Comments"]

        # License and official_video
        data['Licensed'].fillna(data['official_video'], inplace=True)
        data['Licensed'].fillna(False, inplace=True)

        data['official_video'].fillna(data['Licensed'], inplace=True)
        data['official_video'].fillna(False, inplace=True)

        data['official_video'].fillna(False, inplace=True)
        data['Licensed'].fillna(False, inplace=True)

        data['Licensed'] =  data['Licensed'].map({True: 1, False: 0})
        data['official_video'] = data['official_video'].map({True: 1, False: 0})
    
        # Create new class = 'Unknown'
        data['Composer'].fillna("Unknown", inplace=True)
        data['Artist'].fillna("Unknown", inplace=True)
        data['Album_type'].fillna("Unknown", inplace=True)

        newData = self.filterArtistComposerDance(data, nameColumnFill)
        data = newData.copy()

        newData = self.filterArtist(data, nameColumnFill)
        data = newData.copy()

        newData = self.filterFillData(data, nameColumnFill)
        data = newData.copy()

        #Transform type key to use as class
        data['Key'] = data['Key'].astype(int)
        data['Key'] = data['Key'].astype(str)

        data = data.sort_values('id')

        # DELETE Track, Album, Uri, Url_spotify, Url_youtube, Description, Title, Channel, id, Comments
        data.drop(['Track', 'Album', 'Uri', 'Url_spotify', 'Url_youtube', 'Description', 'Title', 'Channel', 'id'], axis=1, inplace=True)

        # pd.options.mode.chained_assignment = 'warn'

        return data
    
    def convertEncoderPD(self, data, prefix = 'key'):
        titleKeys = []
        for i in range(data.shape[1]):
            titleKeys.append(f'{prefix}_{i}')
    
        return pd.DataFrame(data=data, columns= titleKeys)
    
    # minX -60  maxX = 0  ~ 0 - 1
    # y = (-1/60) x
    def scaleMinMaxLoudness(self, data):
        return -data/60
    
    def createEncodeDataTraining(self, data):

        encoderKey = OneHotEncoder()
        encodedKey = encoderKey.fit_transform(data[['Key']])
        Key = encodedKey.toarray()
        key_pd = self.convertEncoderPD(Key, prefix = 'key')

        encoderAlbumType = OneHotEncoder()
        encodedKeyAlbumType = encoderAlbumType.fit_transform(data[['Album_type']])
        AlbumType = encodedKeyAlbumType.toarray()
        AlbumType_pd = self.convertEncoderPD(AlbumType, prefix = 'AlbumType')

        encoderComposer = OneHotEncoder()
        encodedKeyComposer = encoderComposer.fit_transform(data[['Composer']])
        Composer = encodedKeyComposer.toarray()
        Composer_pd = self.convertEncoderPD(Composer, prefix = 'Composer')

        # encoderArtist = OneHotEncoder()
        # encodedArtist = encoderArtist.fit_transform(data[['Artist']])
        # Artist =  encodedArtist.toarray()
        # Artist_pd = self.convertEncoderPD(Artist, prefix = 'Artist')

        encoderArtist = LabelEncoder()
        encodedArtist = encoderArtist.fit_transform(data[['Artist']])
        # encodedArtist = encodedArtist.ravel()
        Artist_pd =  pd.DataFrame(data=encodedArtist, columns= ["Artist"])

        data.drop(['Key','Album_type', 'Composer',  'Artist'], axis=1, inplace=True)

        data = pd.concat([data, key_pd, AlbumType_pd, Composer_pd, Artist_pd], axis=1)

        scaledLoudness = self.scaleMinMaxLoudness(data[['Loudness']])
        data['Loudness'] = scaledLoudness

        newMinMaxScaler = ['Tempo', 'Duration_ms', 'Views', 'Likes', 'Stream', 'Comments']

        # scaler = MinMaxScaler()
        scaler = StandardScaler()
        scaledData = scaler.fit_transform(data[newMinMaxScaler])

        for i in range(scaledData.shape[1]):
            data[newMinMaxScaler[i]] = scaledData[:, i]


        self.dataEncoders = {"key": encoderKey, 'AlbumType': encoderAlbumType, 'Composer': encoderComposer, "Artist":encoderArtist}
        self.stdScaler = scaler
        self.data = data

    # def dataBlocksGenerator(self, data):
    #     musicMarket = pd.concat([data['Energy'], data['Valence'], data['Tempo'], data.iloc[:, 8:15]], axis=1)
    #     musicStats = pd.concat([data.iloc[:, 0:8], data.loc[:, data.columns.str.startswith("key")]], axis=1)       
    #     musicArtistComposer = pd.concat([data['Energy'], data['Valence'], data['Tempo'], data.loc[:, data.columns.str.startswith("key")], data.loc[:, data.columns.str.startswith("Artist")], data.loc[:, data.columns.str.startswith("Composer")], data.loc[:, data.columns.str.startswith("AlbumType")]], axis=1)

    #     return musicMarket, musicStats, musicArtistComposer 


    def dataBlocksGenerator(self, data):
        musicMarket = pd.concat([data.iloc[:, 8:15]], axis=1)
        musicStats = pd.concat([data.iloc[:, 0:8], data.loc[:, data.columns.str.startswith("key")]], axis=1)       
        musicArtistComposer = pd.concat([ data.loc[:, data.columns.str.startswith("Artist")], data.loc[:, data.columns.str.startswith("Composer")], data.loc[:, data.columns.str.startswith("AlbumType")]], axis=1)

        print(musicArtistComposer.shape)
        return musicMarket, musicStats, musicArtistComposer 

    class DataPreparation(Dataset):
        def __init__(
                self, 
                musicMarket, 
                musicStats, 
                musicArtistComposer, 
                labels):
            
            self.musicMarket = torch.from_numpy(np.asarray(musicMarket).astype(np.float32)).to(torch.float32)
            self.musicStats = torch.from_numpy(np.asarray(musicStats).astype(np.float32)).to(torch.float32)
            self.musicArtistComposer = torch.from_numpy(np.asarray(musicArtistComposer).astype(np.float32)).to(torch.float32)
            # self.labels = torch.from_numpy(np.asarray(labels, dtype=np.float32).reshape(labels.shape[0], 1)).to(torch.float32)
            self.labels = torch.from_numpy(np.asarray(labels, dtype=np.float32)).to(torch.float32)
            
            self.n_samples = labels.shape[0]

        def __getitem__(self, index):
            return self.musicMarket[index], \
            self.musicStats[index], \
            self.musicArtistComposer[index], \
            self.labels[index]
        
        def __len__(self):
            return self.n_samples

    def splitDataset(self):
        train_X, test_X, train_Y, test_Y = train_test_split(self.data, self.labels, test_size = 0.20, random_state = 123, shuffle=True)
        train_X, Validation_X, train_Y, Validation_Y = train_test_split(train_X, train_Y, test_size = 0.20, random_state = 123, shuffle=True)

        train_musicMarket, train_musicStats, train_musicArtistComposer = self.dataBlocksGenerator(train_X)
        Validation_musicMarket, Validation_musicStats, Validation_musicArtistComposer = self.dataBlocksGenerator(Validation_X)
        test_musicMarket, test_musicStats, test_musicArtistComposer = self.dataBlocksGenerator(test_X)
        
        self.train = self.DataPreparation(train_musicMarket, train_musicStats, train_musicArtistComposer, train_Y)
        self.validate = self.DataPreparation(Validation_musicMarket, Validation_musicStats, Validation_musicArtistComposer, Validation_Y)
        self.test = self.DataPreparation(test_musicMarket, test_musicStats, test_musicArtistComposer, test_Y)


In [5]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        musicMarket = 7
        musicStats = 19
        musicArtistComposer = 16

        out_musicMarket = 64
        out_musicStats = 64
        out_musicArtistComposer = 64

        concat = out_musicMarket + out_musicStats + out_musicArtistComposer

        # musicMarket
        self.mMl1 = nn.Linear(musicMarket, 16)
        self.mMact1 = nn.ReLU()
        self.mMd1 = nn.Dropout(p=0.2) 

        self.mMl2 = nn.Linear(16, 32)
        self.mMact2 = nn.ReLU()
        self.mMd2 = nn.Dropout(p=0.2) 

        self.mMl3 = nn.Linear(32, 48)
        self.mMact3 = nn.ReLU()
        self.mMd3 = nn.Dropout(p=0.2) 

        self.mMl4 = nn.Linear(48, out_musicMarket)
        self.mMact4 = nn.ReLU()
        self.mMd4 = nn.Dropout(p=0.2) 

        # musicStats

        self.msl1 = nn.Linear(musicStats, 32)
        self.msact1 = nn.ReLU()
        self.msd1 = nn.Dropout(p=0.2)

        self.msl2 = nn.Linear(32, 48)
        self.msact2 = nn.ReLU()
        self.msd2 = nn.Dropout(p=0.2)

        self.msl3 = nn.Linear(48, out_musicStats)
        self.msact3 = nn.ReLU()
        self.msd3 = nn.Dropout(p=0.2)
        


        # music musicArtistComposer
        self.mAl1 = nn.Linear(musicArtistComposer, 32)
        self.mAact1 = nn.ReLU()
        self.mAd1 = nn.Dropout(p=0.2) 

        self.mAl2 = nn.Linear(32, 48)
        self.mAact2 = nn.ReLU()
        self.mAd2 = nn.Dropout(p=0.2) 

        self.mAl3 = nn.Linear(48, out_musicArtistComposer)
        self.mAact3 = nn.ReLU()
        self.mAd3 = nn.Dropout(p=0.2) 

        

        # Concatenation
        self.cCl1 = nn.Linear(concat, 80)
        self.cCact1 = nn.ReLU()
        self.cCd1 = nn.Dropout(p=0.2) 

        self.cCl2 = nn.Linear(80, 64)
        self.cCact2 = nn.ReLU()
        self.cCd2 = nn.Dropout(p=0.2) 

        self.cCl3 = nn.Linear(64, 32)
        self.cCact3 = nn.ReLU()
        self.cCd3 = nn.Dropout(p=0.2) 

        self.cCl4 = nn.Linear(32, 10)
        
        # self.out = nn.Softmax(dim=1)
    
    def forward(self, musicMarket, musicStats, musicArtistComposer):

        # musicMarket
        out1 = self.mMl1(musicMarket)
        out1 = self.mMact1(out1)
        out1 = self.mMd1(out1)

        out1 = self.mMl2(out1)
        out1 = self.mMact2(out1)
        out1 = self.mMd2(out1)

        out1 = self.mMl3(out1)
        out1 = self.mMact3(out1)
        out1 = self.mMd3(out1) 

        out1 = self.mMl4(out1)
        out1 = self.mMact4(out1)
        out1 = self.mMd4(out1)

        # musicStats

        out2 = self.msl1(musicStats)
        out2 = self.msact1(out2)
        out2 = self.msd1(out2)

        out2 = self.msl2(out2)
        out2 = self.msact2(out2)
        out2 = self.msd2(out2)

        out2 = self.msl3(out2)
        out2 = self.msact3(out2)
        out2 = self.msd3(out2)

        # music musicArtistComposer
   
        out3 = self.mAl1(musicArtistComposer)
        out3 = self.mAact1(out3)
        out3 = self.mAd1(out3) 

        out3 = self.mAl2(out3)
        out3 = self.mAact2(out3)
        out3 = self.mAd2(out3) 

        out3 = self.mAl3(out3)
        out3 = self.mAact3(out3)
        out3 = self.mAd3(out3)

        concat = torch.cat((out1, out2, out3), 1)

        out = self.cCl1(concat)
        out = self.cCact1(out)
        out = self.cCd1(out)

        out = self.cCl2(out)
        out = self.cCact2(out)
        out = self.cCd2(out)

        out = self.cCl3(out)
        out = self.cCact3(out)
        out = self.cCd3(out) 

        return self.cCl4(out)
        
      

        # out = self.cCl4(out)
        
        # return self.out(out)

    

In [6]:
# class Model1(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         musicMarket = 10
#         musicStats = 19
#         musicArtistComposer = 126

#         out_musicMarket = 4
#         out_musicStats = 8
#         out_musicArtistComposer = 32

#         concat = out_musicMarket + out_musicStats + out_musicArtistComposer
        
#         # music Market
#         self.mMl1 = nn.Linear(musicMarket, 8)
#         self.mMact1 = nn.ReLU()
#         self.mMd1 = nn.Dropout(p=0.2)

#         self.mMl2 = nn.Linear(8, 6)
#         self.mMact2 = nn.ReLU()
#         self.mMd2 = nn.Dropout(p=0.2)

#         self.mMl3 = nn.Linear(6, out_musicMarket)
#         self.mMact3 = nn.ReLU()
#         self.mMd3 = nn.Dropout(p=0.2)

#         # music Stats
#         self.mSl1 = nn.Linear(musicStats, 16)
#         self.mSact1 = nn.ReLU()
#         self.mSd1 = nn.Dropout(p=0.2) 

#         self.mSl3 = nn.Linear(16, 12)
#         self.mSact3 = nn.ReLU()
#         self.mSd3 = nn.Dropout(p=0.2)

#         self.mSl5 = nn.Linear(12, out_musicStats)
#         self.mSact5 = nn.ReLU()
#         self.mSd5 = nn.Dropout(p=0.2)

#         # music musicArtistComposer
#         self.mAl1 = nn.Linear(musicArtistComposer, 96)
#         self.mAact1 = nn.ReLU()
#         self.mAd1 = nn.Dropout(p=0.2) 

#         self.mAl2 = nn.Linear(96, 64)
#         self.mAact2 = nn.ReLU()
#         self.mAd2 = nn.Dropout(p=0.2) 

#         self.mAl3 = nn.Linear(64, out_musicArtistComposer)
#         self.mAact3 = nn.ReLU()
#         self.mAd3 = nn.Dropout(p=0.2) 

#         # Concatenation
#         self.cCl1 = nn.Linear(concat, 36)
#         self.cCact1 = nn.ReLU()
#         self.cCd1 = nn.Dropout(p=0.2) 

#         self.cCl2 = nn.Linear(36, 16)
#         self.cCact2 = nn.ReLU()
#         self.cCd2 = nn.Dropout(p=0.2) 

#         # self.cCl3 = nn.Linear(16, 4)
#         # self.cCact3 = nn.ReLU()
#         # self.cCd3 = nn.Dropout(p=0.2) 

#         self.cCl4 = nn.Linear(16, 10)
#         self.out = nn.Softmax(dim=1)
    
#     def forward(self, musicMarket, musicStats, musicArtistComposer):
#         # music Market
#         outmM = self.mMl1(musicMarket)
#         outmM = self.mMact1(outmM)
#         outmM = self.mMd1(outmM)

#         outmM = self.mMl2(outmM)
#         outmM = self.mMact2(outmM)
#         outmM = self.mMd2(outmM)

#         outmM = self.mMl3(outmM)
#         outmM = self.mMact3(outmM)
#         outmM = self.mMd3(outmM)

#         # music Stats   
#         outmS = self.mSl1(musicStats)
#         outmS = self.mSact1(outmS)
#         outmS = self.mSd1(outmS) 

#         outmS = self.mSl3(outmS)
#         outmS = self.mSact3(outmS)
#         outmS = self.mSd3(outmS)

#         outmS = self.mSl5(outmS)
#         outmS = self.mSact5(outmS)
#         outmS = self.mSd5(outmS)

#         # music musicArtistComposer
#         outmA = self.mAl1(musicArtistComposer)
#         outmA = self.mAact1(outmA)
#         outmA = self.mAd1(outmA)

#         outmA = self.mAl2(outmA)
#         outmA = self.mAact2(outmA)
#         outmA = self.mAd2(outmA) 

#         outmA = self.mAl3(outmA)
#         outmA = self.mAact3(outmA)
#         outmA = self.mAd3(outmA) 

#         concat = torch.cat((outmM, outmS, outmA), 1)

#         # Concatenation
#         out = self.cCl1(concat)
#         out = self.cCact1(out)
#         out = self.cCd1(out) 

#         out = self.cCl2(out)
#         out = self.cCact2(out)
#         out = self.cCd2(out) 

#         # out = self.cCl3(out)
#         # out = self.cCact3(out)
#         # out = self.cCd3(out) 

#         out = self.cCl4(out)
#         return self.out(out)
#         # return out

In [7]:
# class Model(nn.Module):
#     def __init__(self):
#         super(Model, self).__init__()
#         musicMarket = 10
#         musicStats = 19
#         musicArtistComposer = 126

#         # out_musicMarket = 4
#         # out_musicStats = 8
#         out_musicArtistComposer = 32

#         concat = musicMarket + musicStats + out_musicArtistComposer
        


#         # music musicArtistComposer
#         self.mAl1 = nn.Linear(musicArtistComposer, musicArtistComposer)
#         self.mAact1 = nn.ReLU()
#         self.mAd1 = nn.Dropout(p=0.2) 

#         self.mAl2 = nn.Linear(musicArtistComposer, musicArtistComposer)
#         self.mAact2 = nn.ReLU()
#         self.mAd2 = nn.Dropout(p=0.2) 

#         self.mAl3 = nn.Linear(musicArtistComposer, 94)
#         self.mAact3 = nn.ReLU()
#         self.mAd3 = nn.Dropout(p=0.2) 

#         self.mAl4 = nn.Linear(94, 94)
#         self.mAact4 = nn.ReLU()
#         self.mAd4 = nn.Dropout(p=0.2) 

#         self.mAl5 = nn.Linear(94, 62)
#         self.mAact5 = nn.ReLU()
#         self.mAd5 = nn.Dropout(p=0.2)

#         self.mAl6 = nn.Linear(62, 62)
#         self.mAact6 = nn.ReLU()
#         self.mAd6 = nn.Dropout(p=0.2)

#         self.mAl7 = nn.Linear(62, out_musicArtistComposer)
#         self.mAact7 = nn.ReLU()
#         self.mAd7 = nn.Dropout(p=0.2)

#         # Concatenation
#         self.cCl1 = nn.Linear(concat, concat)
#         self.cCact1 = nn.ReLU()
#         self.cCd1 = nn.Dropout(p=0.2) 

#         self.cCl2 = nn.Linear(concat, concat)
#         self.cCact2 = nn.ReLU()
#         self.cCd2 = nn.Dropout(p=0.2) 

#         self.cCl3 = nn.Linear(concat, concat)
#         self.cCact3 = nn.ReLU()
#         self.cCd3 = nn.Dropout(p=0.2) 

#         self.cCl4 = nn.Linear(concat, concat)
#         self.cCact4 = nn.ReLU()
#         self.cCd4 = nn.Dropout(p=0.2) 

#         self.cCl5 = nn.Linear(concat, concat)
#         self.cCact5 = nn.ReLU()
#         self.cCd5 = nn.Dropout(p=0.2) 

#         self.cCl6 = nn.Linear(concat, 36)
#         self.cCact6 = nn.ReLU()
#         self.cCd6 = nn.Dropout(p=0.2) 

#         self.cCl7 = nn.Linear(36, 36)
#         self.cCact7 = nn.ReLU()
#         self.cCd7 = nn.Dropout(p=0.2) 

#         self.cCl8 = nn.Linear(36, 36)
#         self.cCact8 = nn.ReLU()
#         self.cCd8 = nn.Dropout(p=0.2) 

#         self.cCl9 = nn.Linear(36, 16)
#         self.cCact9 = nn.ReLU()
#         self.cCd9 = nn.Dropout(p=0.2)

#         self.pout = nn.Linear(16, 10)
#         self.out = nn.Softmax(dim=1)
    
#     def forward(self, musicMarket, musicStats, musicArtistComposer):

#         mac = self.mAl1(musicArtistComposer)
#         mac = self.mAact1(mac)
#         mac = self.mAd1(mac) 

#         mac = self.mAl2(mac)
#         mac = self.mAact2(mac)
#         mac = self.mAd2(mac)

#         mac = self.mAl3(mac)
#         mac = self.mAact3(mac)
#         mac = self.mAd3(mac) 

#         mac = self.mAl4(mac)
#         mac = self.mAact4(mac)
#         mac = self.mAd4(mac) 

#         mac = self.mAl5(mac)
#         mac = self.mAact5(mac)
#         mac = self.mAd5(mac)

#         mac = self.mAl6(mac)
#         mac = self.mAact6(mac)
#         mac = self.mAd6(mac)

#         mac = self.mAl7(mac)
#         mac = self.mAact7(mac)
#         mac = self.mAd7(mac)


#         concat = torch.cat((musicMarket, musicStats, mac), 1)

#         # Concatenation
#         out = self.cCl1(concat)
#         out = self.cCact1(out)
#         out = self.cCd1(out)

#         out = self.cCl2(out)
#         out = self.cCact2(out)
#         out = self.cCd2(out)

#         out = self.cCl3(out)
#         out = self.cCact3(out)
#         out = self.cCd3(out) 

#         out = self.cCl4(out)
#         out = self.cCact4(out)
#         out = self.cCd4(out)

#         out = self.cCl5(out)
#         out = self.cCact5(out)
#         out = self.cCd5(out) 

#         out = self.cCl6 (out)
#         out = self.cCact6(out)
#         out = self.cCd6(out) 

#         out = self.cCl7(out)
#         out = self.cCact7(out)
#         out = self.cCd7(out) 

#         out = self.cCl8(out)
#         out = self.cCact8(out)
#         out = self.cCd8(out) 

#         out = self.cCl9(out)
#         out = self.cCact9(out)
#         out = self.cCd9(out)

#         # return self.pout(out)

#         out = self.pout(out)
        
#         return self.out(out)

In [18]:
class DLModel():
    def __init__(self, train, test, validate, learning_rate=1e-6, epoch = 10, batch_size = 16, device = 'cpu'):
        self.lr = learning_rate
        self.epoch = epoch
        self.batch_size = batch_size
        self.device = self.getDevice(device)

        self.train = DataLoader(dataset = train, batch_size = self.batch_size, shuffle = True)
        self.validation = DataLoader(dataset = validate, batch_size = self.batch_size, shuffle = False)
        self.test = DataLoader(dataset = test, batch_size = self.batch_size, shuffle = False)
        
        self.model = Model().to(self.device)
        # Define the loss function
        # self.criterion = nn.MSELoss()
        self.criterion = nn.CrossEntropyLoss()
        # Create an SGD optimizer with L1 regularization
        # self.optimizer = torch.optim.SGD(self.model.parameters(), lr=self.lr, weight_decay=0.001)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        
    
    def getDevice(self, device):
        if device != 'cpu':
            is_cuda =  torch.cuda.is_available()
            if is_cuda:
                return torch.device('cuda')
            return torch.device('cpu')
        return torch.device('cpu')
    
    def metric(self, y, y_hat):
        return np.mean(np.abs(y_hat - y))

    def trainModel(self, process_factor=2, patiente = 2, show_process = False, earlyStopFlag=True):
        self.lossTraining = []
        self.accTraining = []
        self.lossValidation = []
        self.accValidation  = []

        patiente_acum = 0  # patiente counter

        prev_metric = float('inf')  # best loss from validation
        best_Model =  copy.deepcopy(self.model.state_dict()) # best model
        self.best_epoch = 0

        process = tqdm(range(self.epoch),  desc = 'Epoch ===> ')
        for epoch in process:
            y_hat = []
            y = []
            flag = True

            loss_train = 0.0
            acc_train  = 0.0

            self.model.train()
            for idx, (musicMarket, musicStats, musicArtistComposer, labels) in enumerate(self.train):
                musicMarket = musicMarket.to(self.device)
                musicStats = musicStats.to(self.device)
                musicArtistComposer = musicArtistComposer.to(self.device)
                labels = labels.to(self.device)

                # forward
                outputs = self.model(musicMarket, musicStats, musicArtistComposer)
            

                loss = self.criterion(outputs, labels)

                
                # backward
                # loss.backward()
                # self.optimizer.step()
                # self.optimizer.zero_grad()
                
        


                # optimizer.zero_grad()
                # loss.backward()
                # # update weights
                # optimizer.step()
                
                '''
                    ---------------  Get predicted to calculate accuracy ----------------
                '''
                

                predicted = outputs.cpu().detach().numpy()
                ground_truth = labels.cpu().detach().numpy()

                


                predictedMax = np.argmax(predicted, 1)
                groundTruthMax = np.argmax(ground_truth, 1)

                if flag:
                    y_hat = predictedMax
                    y = groundTruthMax
                    flag = False

                y_hat = np.append(y_hat, predictedMax)
                y = np.append(y, groundTruthMax)


                
                # acc = np.mean(y_hat == y)
                mae = self.metric(y, y_hat)

                if show_process:
                    if (idx + 1) % process_factor== 0:
                        text = f'Training Stage ==> Epoch: {epoch} / {self.epoch - 1} | Step: {idx} / {len(self.train)} | Training loss: {loss.item():.5f} |  Training Evaluation Metric (MAE): {mae:.5f}'
                        print(text)

                loss_train = loss.item()
                acc_train = mae

                process.set_postfix({'Epoch':epoch,
                    'training_loss': loss_train, 
                    'training Acc': acc_train, 
                    'Step': idx,
                    'from': len(self.train)
                    })
            
            self.lossTraining.append(loss_train)
            self.accTraining.append(acc_train)
            metric = self.metric(y, y_hat)
            acc = np.mean(y_hat == y)

            text = f'Training Stage ==> Epoch: {epoch} / {self.epoch - 1} | Training loss: {loss_train:.5f} |  Training Accuracy: {acc:.5f} | Training Metric (MAE): {metric:.5f} | Acc: {acc * 100}'
            print(text)
                


            ''' 
            ----------------------------------  EVALUATION STAGE  ---------------------------------------
                
            '''
            y_hat = []
            y = []
            flag = True

            loss_val = 0.0
            acc_val  = 0.0
      
      
            self.model.eval()
            with torch.no_grad():
                for idx, (musicMarket, musicStats, musicArtistComposer, labels) in enumerate(self.validation):
                    musicMarket = musicMarket.to(self.device)
                    musicStats = musicStats.to(self.device)
                    musicArtistComposer = musicArtistComposer.to(self.device)
                    labels = labels.to(self.device)

                    '''
                    ---------------   Forward    --------------------
                    '''

                    outputs = self.model(musicMarket, musicStats, musicArtistComposer)
                    loss = self.criterion(outputs, labels)


                    predicted = outputs.cpu().detach().numpy()
                    ground_truth = labels.cpu().detach().numpy()
                    
                    predictedMax = np.argmax(predicted, 1)
                    groundTruthMax = np.argmax(ground_truth, 1)

                    if flag:
                        y_hat = predictedMax
                        y = groundTruthMax
                        flag = False

                    y_hat = np.append(y_hat, predictedMax)
                    y = np.append(y, groundTruthMax)


                
                    acc = np.mean(y_hat == y)
                    # acc = self.metric(y, y_hat)

                    if show_process:
                        if (idx + 1) % process_factor== 0:
                            text = f'Validation Stage ==> Epoch: {epoch} / {self.epoch - 1} | Step: {idx} / {len(self.validation)} | Validation loss: {loss.item():.5f} |  Validation Evaluation Metric (MAE): {acc:.5f}'
                            print(text)

                    loss_val = loss.item()
                    acc_val = acc
                    process.set_postfix({'Epoch':epoch,
                    'validation_loss': loss_val, 
                    'Validation Acc': acc_val, 
                    'Step': idx,
                    'from': len(self.validation)
                    })
            
                self.lossValidation.append(loss_val)
                self.accValidation.append(acc_val)
                metric = self.metric(y, y_hat)
                acc = np.mean(y_hat == y)

                text = f'Validation Stage ==> Epoch: {epoch} / {self.epoch - 1} | Validation loss: {loss_val:.5f} |  Validation Accuracy: {acc:.5f} | Validation Metric (MAE)): {metric:.5f} | Acc: {acc * 100}'
                print(text)

                if metric < prev_metric:
                    prev_metric = metric
                    best_Model =  copy.deepcopy(self.model.state_dict())
                    torch.save(self.model, f'./model/model_EPOCH_{epoch}')

                    # np.save(f'./training/lossTraining_EPOCH_{epoch}',np.asarray(self.lossTraining))
                    # np.save(f'./training/accTraining_EPOCH_{epoch}',np.asarray(self.accTraining))
                    # np.save(f'./training/lossValidation_EPOCH_{epoch}',np.asarray(self.lossValidation))
                    # np.save(f'./training/accValidation_EPOCH_{epoch}',np.asarray(self.accValidation))
          
    
                    self.best_epoch = epoch
                    patiente_acum = 0
                else:
                    patiente_acum += 1


                if earlyStopFlag and patiente_acum > patiente:
                    # Load best model in current model
                    self.model.load_state_dict(best_Model)
          
                    print(f'Early Stop Load Model from best epoch {self.best_epoch}')
                    break


            # print(earlyStopFlag)
            # if ~earlyStopFlag:
            #     print("CAGAdAAAA")
            #     # Load best model in current model
            #     self.model.load_state_dict(best_Model)
            #     print(f'Load Model from best epoch {self.best_epoch}') 

            process.update(1)
        process.close()



In [19]:
f = DataFinal()

(10988, 16)
(2748, 16)
(3434, 16)


c:\Users\Miguel\anaconda3\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [20]:
f.data

,Energy,Loudness,Speechiness,Acousticness,Instrumentalness,Liveness,Valence,Tempo,Duration_ms,Views,...,Composer_2,Composer_3,Composer_4,Composer_5,Composer_6,Composer_7,Composer_8,Composer_9,Composer_10,Artist
0,0.000273,0.419600,0.0443,7.241508e-01,0.000062,0.000807,0.3400,-1.255897,-0.929988,-0.152250,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,59
577,0.184220,0.229350,0.0340,6.722214e-01,0.910000,0.034966,0.7460,0.944228,-0.447282,-0.352389,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,34
910,0.384241,0.259933,0.0442,6.676276e-01,0.867000,0.001772,0.3800,-1.205320,-0.645164,-0.352389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,85
1121,0.209585,0.104183,0.0277,3.796416e-03,0.000000,0.001000,0.5110,0.995715,0.348878,0.494190,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79
1367,0.354895,0.082267,0.0260,3.048625e-06,0.000467,0.002924,0.2950,-0.763974,0.339028,0.972284,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13828,0.794023,0.105733,0.0328,8.991539e-02,0.000000,0.000591,0.6580,-1.033371,-1.132043,-0.121606,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,34
11326,0.820026,0.029767,0.1610,2.195200e-05,0.000000,0.000786,0.6570,1.826279,-0.643512,-0.352473,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43
4096,0.571787,0.077983,0.0647,5.751246e-02,0.000000,0.003652,0.4190,1.607898,-0.765362,-0.353013,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,91
15904,0.451218,0.196533,0.4190,4.511802e-02,0.000000,0.001260,0.5390,1.169517,-1.012757,-0.353135,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,73


In [25]:
t = DLModel(f.train, f.validate, f.test, learning_rate=1e-2, epoch = 1000, batch_size = 32)

In [26]:
t.trainModel(earlyStopFlag =False)

Epoch ===> :   0%|          | 0/1000 [00:01<?, ?it/s, Epoch=0, validation_loss=2.29, Validation Acc=0.111, Step=53, from=108] 

Training Stage ==> Epoch: 0 / 999 | Training loss: 2.32352 |  Training Accuracy: 0.09574 | Training Metric (MAE): 3.19764 | Acc: 9.573502722323049


Epoch ===> :   0%|          | 2/1000 [00:02<35:06,  2.11s/it, Epoch=1, training_loss=2.31, training Acc=3.22, Step=18, from=344]       

Validation Stage ==> Epoch: 0 / 999 | Validation loss: 2.32410 |  Validation Accuracy: 0.09954 | Validation Metric (MAE)): 3.18321 | Acc: 9.953837276399307


Epoch ===> :   0%|          | 2/1000 [00:03<35:06,  2.11s/it, Epoch=1, validation_loss=2.34, Validation Acc=0.111, Step=51, from=108] 

...

Training Stage ==> Epoch: 21 / 999 | Training loss: 2.29987 |  Training Accuracy: 0.09465 | Training Metric (MAE): 3.21897 | Acc: 9.464609800362975


Epoch ===> :   3%|▎         | 29/1000 [00:42<30:45,  1.90s/it, Epoch=22, training_loss=2.33, training Acc=3.18, Step=40, from=344]       

Validation Stage ==> Epoch: 21 / 999 | Validation loss: 2.32410 |  Validation Accuracy: 0.09954 | Validation Metric (MAE)): 3.18321 | Acc: 9.953837276399307


Epoch ===> :   2%|▏         | 22/1000 [00:42<31:38,  1.94s/it, Epoch=22, training_loss=2.31, training Acc=3.2, Step=165, from=344] 


KeyboardInterrupt: 